# Techniques for sampling parameter space - Grid Sampling

Another sampling technique is 'grid sampling', where samples are taken at regular grid intervals across the parameter space.

This is an example of 'biased sampling' because the samples are ordered according to the grid specified.

The following code runs a neutronics simulation using a simple pre-defined model. The code varies the input parameters according to a specified grid. 
Li6 enrichment and breeder percent in breeder plus multiplier ratio are both varied between 0 and 100.

In [ ]:
from pathlib import Path
import json
import uuid
import math
import argparse

import numpy as np
from tqdm import tqdm

from openmc_model import find_tbr_hcpb
from plot_sampling_coordinates import plot_simulation_results, read_in_data
from new_interpolated_results import plot_interpolated_results


number_of_simulations = 16

number_of_steps = math.floor(math.sqrt(number_of_simulations))


print("running simulations with grid sampling")

for breeder_percent_in_breeder_plus_multiplier_ratio in tqdm(np.linspace(0.1, 100, number_of_steps)):
    for blanket_breeder_li6_enrichment in np.linspace(0.1, 100, number_of_steps):

        result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                               blanket_breeder_li6_enrichment)
        
        result["sample"] = "grid"
        
        filename = "outputs/" + str(uuid.uuid4()) + ".json"
        Path(filename).parent.mkdir(parents=True, exist_ok=True)
        with open(filename, mode="w", encoding="utf-8") as f:
            json.dump(result, f, indent=4)

print("results saved in output file")

results_df = read_in_data()
filtered_results_df = results_df[results_df["sample"] == "grid"]

In [ ]:
# plot results

plot_simulation_results(filtered_results_df)

In [ ]:
# plot interpolated results

plot_interpolated_results(filtered_results_df)

Grid sampling maximises the spatial distribution of sample points across the parameter space by avoiding point clustering and, therefore, maximises the amount of useful information obtained from each simulation. As a result, grid sampling is a highly efficient sampling technique for covering a parameter space.

However, as simulations are performed in order according to the defined grid, they are biased towards the first parameter values in the grid, I.e. they are biased towards one area of parameter space. For example, the code abbove performs simulations with breeder percent in breeder plus multiplier ratio = 0 for all Li6 enrichments before breeder percent in breeder plus multiplier ratio is changed. This means that all simulations across the grid must be performed before a data trend across the entire parameter space can be observed. This is the main disadvantage of grid sampling as more simulations than are necessary may be performed, and it is difficult to efficiently add sample points to the existing data without performing a complete new grid search 'in-between' the existing data points. On the other hand, random sampling is unbiased meaning the data trend across the parameter space can be observed with a small number of simulations and additional sample points can be added easily.

Overall, grid sampling is a more efficient sampling technique than random sampling, but its bias towards initial parameter values makes it unsuitable for most applications. Instead, we tend to use more advanced 'quasi-random' or 'adaptive' techniques to improve sampling efficiency.